In [ ]:
from fastecdsa.curve import secp256k1
import fastecdsa.keys as keys
import fastecdsa.point as Point
import ecc
import util
from hashlib import sha256
from os import urandom
# Borromean signatures
# Pedersen Commitments

In [ ]:
# Borromean Signatures
## First we have a set of participants, each with his own key pair
participants = {'A': 0, 'B': 0, 'C': 0, 'D': 0}

for p in participants:
    participants[p] = {'Keys': ecc.PrivateKey(int.from_bytes(urandom(32), 'little'))}

In [ ]:
## Now let's say one of the participant, B, want to make a signature
## First, he needs to choose a new nonce key pair
participants['B'].update({'Nonce': ecc.PrivateKey(int.from_bytes(urandom(32), 'little'))})
## Then we need to create a message hash for the next participant. 
## This contains the message itself (here the pubkey we commit to), the public nonce K and an index
def create_hash_message(K, m, n):
    """generate a hash of the nonce K, the message m and some index, 
    here the letter of the participant
    """
    if isinstance(K, bytes) and isinstance(m, bytes) and isinstance(n, bytes):
        to_hash = K + m + n
        return sha256(to_hash).digest()
    else:
        raise TypeError('K, m and n need to be bytes')

K_B = participants['B']['Nonce'].point
print(f'K_B is {K_B.sec().hex()}')
P_B = participants['B']['Keys'].point
m = P_B.sec()
print(f'm is {m.hex()}')
e_C = create_hash_message(K_B.sec(), m, b'C')
print(f'e_C is {e_C.hex()}')

In [ ]:
## Then we choose a random value for C, it will be its "signature" (but it isn't a signature obviously)
def generate_random_sig():
    return int.from_bytes(urandom(32), 'little')
s_C = generate_random_sig()
## With this s, the hashed message e and C pubkey, we can compute a value for C's K
def compute_K(s, G, e, P):
    if isinstance(P, ecc.S256Point) == False:
        raise TypeError('P is not a valid S256Point')
    if isinstance(s, int) and isinstance(e, int):
        return s * G - e * P
    else:
        raise TypeError('s and e must be int')

P_C = participants['C']['Keys'].point
e = int.from_bytes(e_C, 'little')
K_C = compute_K(s_C, ecc.G, e, P_C)
print(f'K_C is {K_C.sec().hex()}')

In [ ]:
## we can now compute e, the hash of the message, for the next participant
e_D = create_hash_message(K_C.sec(), m, b'D')
## And repeat the same operations, first choose a random signature S
s_D = generate_random_sig()

P_D = participants['D']['Keys'].point
e = int.from_bytes(e_D, 'little')
K_D = compute_K(s_D, ecc.G, e, P_D)
print(f'K_D is {K_D.sec().hex()}')

In [ ]:
e_A = create_hash_message(K_D.sec(), m, b'A')

s_A = generate_random_sig()

P_A = participants['A']['Keys'].point

e = int.from_bytes(e_A, 'little')
K_A = compute_K(s_A, ecc.G, e, P_A)
print(f'K_A is {K_A.sec().hex()}')

In [ ]:
## Back to the signer B, we calculate our own hash e
e_B = create_hash_message(K_A.sec(), m, b'B')

## Now we make a real signature using the nonce we created in the first step
e = int.from_bytes(e_B, 'little')
x = participants['B']['Keys'].secret # B private key
k = participants['B']['Nonce'].secret # B private key nonce
print(type(x))

## We need K_B == s_B * G - e * P_B
## we can arrange the equation to look like s_B == e * x + k
s_B = (e * x + k)
print(len(hex(s_B)) // 2)
print(f'K_B is {K_B.sec().hex()}')
assert K_B == compute_K(s_B, ecc.G, e, P_B)
print(s_B.to_bytes(65, 'little').hex())

In [ ]:
## We then publish a set containing one e value and all the signatures s values
## the whole set to verify is then equal to the number of participants + 1, each value is of 32B
rs = {'e_B': e_B, 's_D': s_D, 's_A': s_A, 's_B': s_B, 's_C': s_C}
e = int.from_bytes(rs['e_B'], 'little')
## To verify the signature we start from the given e value, compute all the e value from here until 
## we circle back to our first hash. If we got the same hash e, the signature is valid
vK_B = compute_K(rs['s_B'], ecc.G, e, P_B)
print(f'vK_B : {vK_B.sec().hex()}')
assert K_B == vK_B

e_C = create_hash_message(vK_B.sec(), m, b'C')
vK_C = compute_K(rs['s_C'], ecc.G, e, P_C)
print(f'vK_C : {vK_C.sec().hex()}')
assert K_C == vK_C

e_D = create_hash_message(vK_C, m, b'D')
vK_D = compute_K(rs['s_D'], ecc.G, e, P_D)
print(f'vK_D : {vK_D.sec().hex()}')
assert K_D == vK_D

e_A = create_hash_message(vK_D, m, b'A')
vK_A = compute_K(rs['s_A'], ecc.G, e, P_A)
print(f'vK_A : {vK_A.sec().hex()}')
assert K_A == vK_A

#res = sha256(vK_C.sec() + m + b'D').digest()
#assert res == e